## Requirements do NoteBook
O notebook precisa de algumas bibliotecas especificas. É importante melhorar esta parte já que algumas não estão instalando corretamente.

In [ ]:
!pip3 install awswrangler
!pip3 install numpy==1.16.4
!pip3 install mlxtend
!pip3 install lifetimes
!pip3 install turicreate==6.0
!pip3 install -e ds_colombia
#!conda install numpy --force-reinstall -y

In [ ]:
import pandas as pd
import numexpr as ne
import awswrangler as wr
import sys
import logging
import json
from datetime import datetime
from datetime import date
sys.path.insert(0, 'src/')
from utils.mba_generator import mba_generator
from utils.clv_clusters_missing_products import clv_cluster_missing_products
from utils.anomaly_detector import anomaly_detector
from utils.recommender_generator import recommender
from utils.data_cleaning import mba_data_cleaning
from ds_colombia.ds_colombia.clv.clv_churn import df_to_clv, create_transactional_table, create_summary_table

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

with open('clients/clients.json') as json_file:
    data = json.load(json_file)


In [ ]:
report_date = str(datetime.now())[:10]
bucket = 'fnc-data-science'
products_path = f"test-colombia/products"
model_path = f"test-colombia/models"
view_path = data['comfandi']['view_path']
client_columns = data['comfandi']['extract_columns']

# Seleção das variáveis para a análise

In [ ]:
columns = [client_columns['product_granularity'],
           client_columns['product_name'],
           client_columns['product_group'],
           client_columns['date_column'],
           client_columns['quantity_column'],
           client_columns['order_column'],
           client_columns['value_column'],
           client_columns['customer_column'],
           client_columns['store_granularity'],
           client_columns['anomaly_product_granularity'],
           client_columns['customer_type'],
           client_columns['customer_group'],
           client_columns['customer_category'],
           client_columns['customer_salary'],
           client_columns['customer_city'],
           client_columns['customer_gender'],
           client_columns['customer_age'],
           client_columns['customer_family'],
           client_columns['product_brand'],
           client_columns['product_class'],
           client_columns['product_presentation'],
           client_columns['product_sub_family'],
           client_columns['product_family']]

# Carregamos os dados para a análise

In [ ]:
logging.info("Reading files. Dont't mind the logging overload")
df = wr.s3.read_parquet(view_path, columns=columns)

logging.info("Let's run the MBA stuff")
allowed_product_types = list(
    pd.read_csv('clients/comfandi/comfandi_product_categories.csv')['Allowed Categories'].values)

df = df[df[client_columns['product_granularity']].isin(allowed_product_types)]

In [ ]:
#df.to_parquet('comfandi.parquet')
df = pd.read_parquet('comfandi.parquet')

In [ ]:
import datetime 
#max(df['sale_date'])
#min(df['sale_date'])
#df = df[(df['sale_date'].dt.date > datetime.date(2021,10,1))] 
#df.head()
#df[df['product_sub_family']==' Dermocosmeticos Otc ']
#df = df[df['product_sub_family'].str.contains("Dermocosmeticos Otc")]
#df.head()

## Função que roda o MBA

In [26]:
mba_generator(df,
              bucket,
              products_path,
              report_date,
              client_columns['product_granularity'],
              client_columns['product_name'],
              client_columns['product_group'],
              client_columns['date_column'],
              client_columns['quantity_column'],
              client_columns['order_column'],
              client_columns['value_column'],
              n_products=300,
              n_rules=500,
              #min_support=0.01,
              min_support=0.002,
              max_len=2,
              allowed_product_types=allowed_product_types,
              customer='comfandi')

INFO:root:Cleaning data
INFO:root:Calculating MBA. This is gonna take a while
INFO:root:Entrando na função filter_top_products
INFO:root:Entrando na função prepare_dataset
INFO:root:Entrando na função create_rules_table
INFO:root:Entrando na função fpgrowth
INFO:root:Entrando na função association_rules
INFO:root:Entrando na função create_recommendation_table
INFO:root:Preparing MBA output tables
INFO:root:Saving MBA reports (mba_rules and mba_recommendations) and exclusion report
DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x7fb8a3638268>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x7fb8a36ba268>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x7fb8a8058510>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x7fb8a36ba048>
DEBUG:botocore.end

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning:

indexing past lexsort depth may impact performance.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning:

Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/



DEBUG:botocore.awsrequest:100 Continue response seen, now sending request body.
DEBUG:urllib3.connectionpool:https://fnc-data-science.s3.amazonaws.com:443 "PUT /test-colombia/products/comfandi/2022-12-19/2022-12-19_comfandi_mba_rules.xlsx HTTP/1.1" 200 0
DEBUG:botocore.parsers:Response headers: {'x-amz-id-2': 'Lif7532Hx8uuRdwRkGzGWs+oOINOg9TNDe1h5mmISb3f9m5OHX30IUuAm0yHYyxxAmQQCOF87G0=', 'x-amz-request-id': 'CKPZKW7SZB7ZD6R8', 'Date': 'Mon, 19 Dec 2022 18:32:48 GMT', 'x-amz-server-side-encryption': 'aws:kms', 'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-1:997206602955:key/6f8bf03f-a3f3-450e-8144-94b73ec07483', 'ETag': '"c9529156f4c97b780f50292277f5757c"', 'Server': 'AmazonS3', 'Content-Length': '0'}
DEBUG:botocore.parsers:Response body:
b''
DEBUG:botocore.hooks:Event needs-retry.s3.PutObject: calling handler <botocore.retryhandler.RetryHandler object at 0x7fae284f64a8>
DEBUG:botocore.retryhandler:No retry needed.
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjec

## Função que roda o CLV

In [ ]:
 logging.info("Now it's the CLV and missing products turn")
 end_date = date.today().strftime("%Y-%m-%d")
 clv_cluster_missing_products(df,
                              bucket,
                              products_path,
                              report_date,
                              client_columns['product_granularity'],
                              client_columns['product_name'],
                              client_columns['date_column'],
                              client_columns['quantity_column'],
                              client_columns['order_column'],
                              client_columns['customer_column'],
                              client_columns['store_granularity'],
                              client_columns['value_column'],
                              end_date,
                              period_months=12,
                              customer='comfandi') 

## Funçao que roda o detector de anomalias

In [ ]:
logging.info("Anomalies time!")
anomaly_detector(df,
                  bucket,
                  products_path,
                  report_date,
                  client_columns['anomaly_product_granularity'],
                  client_columns['product_name'],
                  client_columns['date_column'],
                  client_columns['quantity_column'],
                  client_columns['order_column'],
                  client_columns['customer_column'],
                  client_columns['store_granularity'],
                  stddev=2,
                  customer='comfandi')

## Funçao que roda o recommender

In [ ]:
 logging.info("Finally, recommendations to our special users")
 ne.set_num_threads(ne.detect_number_of_threads())
 sales_columns = [client_columns['product_granularity'],
                  client_columns['customer_column']]

 customers_columns = [client_columns['customer_column'],
                      client_columns['customer_type'],
                      client_columns['customer_group'],
                      client_columns['customer_category'],
                      client_columns['customer_salary'],
                      client_columns['customer_city'],
                      client_columns['customer_gender'],
                      client_columns['customer_age'],
                      client_columns['customer_family']]

 products_columns = [client_columns['product_granularity'],
                     client_columns['product_brand'],
                     client_columns['product_class'],
                     client_columns['product_presentation'],
                     client_columns['product_sub_family'],
                     client_columns['product_family']]

In [ ]:
 recommender(df,
             bucket,
             products_path,
             model_path,
             report_date,
             sales_columns,
             customers_columns,
             products_columns,
             customer='comfandi',
             n_recommendations=10,
             customer_column=client_columns['customer_column'],
             product_granularity=client_columns['product_granularity'],
             product_name=client_columns['product_name'],
             quantity_column=client_columns['quantity_column'],
             enhance_diversity=False)